# Usando dados estruturados externos (próprios) nos modelos
Aqui, iremos entender como utilizar bases de dados para alimentar o modelo e ter respostas mais específicas e atualizadas.

A mudança em relação ao que vimos anteriormente é apenas traduzir um banco de dados estruturado em uma linguagem que o modelo poderá entender.
Ou seja, como alterar a função que passaremos como `tool`.

In [63]:
# preparando ambiente
import openai
from dotenv import load_dotenv, find_dotenv
import json
import yfinance as yf # biblioteca da api de dados financeiros da yahoo
# import pandas as pd

# client
_ = load_dotenv(find_dotenv())
client = openai.Client()

## Importando dados financeiros
Antes de mais nada, vamos entender como funciona a biblioteca `yfinance`.

As ações são identificados por códigos (por exemplo PETR4, da Petrobras), utilizamos esse código para importar dados dessas ações:

In [13]:
# objeto da classe ticker
ticker_obj = yf.Ticker(f'{'PETR4'}.SA')
ticker_obj

yfinance.Ticker object <PETR4.SA>

Podemos usar o método `.history()` para pegar os dados históricos, do último mês, por exemplo. Esse método devolve um df:

In [51]:
hist = ticker_obj.history(period='1mo')
type(hist)

pandas.core.frame.DataFrame

In [52]:
hist.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-08-19 00:00:00-03:00,37.531854,37.716599,37.337391,37.376282,28138200,0.0000,0.0
2024-08-20 00:00:00-03:00,37.434622,37.609642,36.841502,37.230434,27322600,0.0000,0.0
2024-08-21 00:00:00-03:00,37.609644,37.716601,36.890120,37.006802,38491900,0.0000,0.0
2024-08-22 00:00:00-03:00,37.279999,37.299999,36.860001,37.119999,26334300,1.0532,0.0
2024-08-23 00:00:00-03:00,37.369999,37.529999,36.889999,36.889999,29239000,0.0000,0.0


In [53]:
hist.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], dtype='object')

In [54]:
print(min(hist.index),max(hist.index))

2024-08-19 00:00:00-03:00 2024-09-19 00:00:00-03:00


Para simplificar, vamos trabalhar apenas com os valores de 'Close':

In [55]:
hist = hist['Close']
hist.head()

Date
2024-08-19 00:00:00-03:00    37.376282
2024-08-20 00:00:00-03:00    37.230434
2024-08-21 00:00:00-03:00    37.006802
2024-08-22 00:00:00-03:00    37.119999
2024-08-23 00:00:00-03:00    36.889999
Name: Close, dtype: float64

In [56]:
# alterando o formato das datas do index
hist.index = hist.index.strftime('%Y-%m-%d')
hist.head()

Date
2024-08-19    37.376282
2024-08-20    37.230434
2024-08-21    37.006802
2024-08-22    37.119999
2024-08-23    36.889999
Name: Close, dtype: float64

In [57]:
# arredondando os valores
hist = round(hist, 2)
hist.head()

Date
2024-08-19    37.38
2024-08-20    37.23
2024-08-21    37.01
2024-08-22    37.12
2024-08-23    36.89
Name: Close, dtype: float64

Transformando o df em um json (uma string de um dicionário com "chave":valor - `"data":preço`)

In [59]:
hist = hist.to_json()
hist

'{"2024-08-19":37.38,"2024-08-20":37.23,"2024-08-21":37.01,"2024-08-22":37.12,"2024-08-23":36.89,"2024-08-26":39.57,"2024-08-27":39.04,"2024-08-28":39.6,"2024-08-29":39.33,"2024-08-30":39.37,"2024-09-02":39.0,"2024-09-03":38.53,"2024-09-04":38.54,"2024-09-05":38.3,"2024-09-06":37.55,"2024-09-09":37.96,"2024-09-10":37.33,"2024-09-11":37.29,"2024-09-12":36.87,"2024-09-13":36.7,"2024-09-16":37.21,"2024-09-17":37.04,"2024-09-18":36.15,"2024-09-19":36.27}'

In [60]:
type(hist)

str

## Criando a função para ser uma tool do modelo
Agora que entendemos como os dados são fornecidos pela API e os transformamos em um json, precisamos criar uma função que retorne os valores históricos quando necessário.

In [64]:
# 1. Criar as funções de apoio 
# usando a api do yahoo para conseguir dados financeiros:
def retorna_cotacao_acao_historica(ticker,periodo='1mo'):
    ticker_obj = yf.Ticker(f'{ticker}.SA')
    hist = ticker_obj.history(period=periodo)['Close']
    hist.index = hist.index.strftime('%Y-%m-%d')
    hist = round(hist, 2)
    # diminuindo o dataframe
    if len(hist) > 30:
        slice_size = int(len(hist) / 30)
        hist = hist.iloc[::-slice_size][::-1]
    return hist.to_json()

# 2. Adicionar as funções à uma lista `tools`
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'retorna_cotacao_acao_historica',
            'description': 'Retorna a cotação diária histórica para uma ação da bovespa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3" para ambev, "PETR4" para petrobras, etc'
                    },
                    'periodo': {
                        'type': 'string',
                        'description': 'O período que será retornado de dados históriocos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',
                        'enum': ["1d","5d","1mo","6mo","1y","5y","10y","ytd","max"]
                    }
                }
            }
        }
    }

]

# 3. Criar um dicionário com `function_name:function`
funcoes_disponiveis = {'retorna_cotacao_acao_historica': retorna_cotacao_acao_historica}

## Testando
Vamos usar o resumo do notebook anterior

In [77]:
# 4. Criar o objeto `messages` com a mensagem inicial
messages = [{'role':'user','content':'Qual é a cotação da petrobras agora?'}]

# 5. Fazer a requisição com o parâmetro `tools`
resposta = client.chat.completions.create(
    model="gpt-4o-mini-2024-07-18",
    tools=tools,
    tool_choice='auto',
    messages=messages,
    max_tokens=100, 
    temperature=0,
)

message_resp = resposta.choices[0].message
tool_calls = message_resp.tool_calls

# 6. Se houver solicitação do modelo para rodar funções:
if tool_calls:
    # 6.1 Adicionar a resposta ao `messages`
    messages.append(message_resp)
    # 6.2 Rodar as funções para cada chamada `tool_calls` solicitada pelo modelo e ...
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(**function_args)
        # ... adicionar as respostas ao `messages`
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )


# 7. Refazer a requisição
nova_resposta = client.chat.completions.create(
        model="gpt-4o-mini-2024-07-18",
        messages=messages,
        max_tokens=100, 
        temperature=0,
    )

# 8. Ler a resposta
print(nova_resposta.choices[0].message.content)

A cotação da ação da Petrobras (PETR4) agora é de R$ 36,27.
